# Build and test the crop container

This notebook is linked to https://eoap.github.io/mastering-app-package/containers/crop/

## Goal

Create a container and run the crop step in the container image.


## Setup the environment

In [1]:
export WORKSPACE=/workspace/mastering-app-package
export RUNTIME=${WORKSPACE}/runs
rm -fr ${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

## Build the container

Inspect the container file:

In [2]:
cat ${WORKSPACE}/water-bodies/command-line-tools/crop/Dockerfile

FROM docker.io/library/python:3.10-slim@sha256:80619a5316afae7045a3c13371b0ee670f39bac46ea1ed35081d2bf91d6c3dbd

RUN pip install --no-cache-dir rasterio click pystac loguru pyproj shapely && \
    python -c "import rasterio"

ADD app.py /app/app.py

ENTRYPOINT []


Build the container using `podman`:

In [3]:
podman build --format docker -t localhost/crop:latest ${WORKSPACE}/water-bodies/command-line-tools/crop


STEP 1/4: FROM docker.io/library/python:3.10-slim@sha256:80619a5316afae7045a3c13371b0ee670f39bac46ea1ed35081d2bf91d6c3dbd
Trying to pull docker.io/library/python@sha256:80619a5316afae7045a3c13371b0ee670f39bac46ea1ed35081d2bf91d6c3dbd...
Getting image source signatures
Copying blob a2318d6c47ec [=>---------------------------------] 1.9MiB / 27.8MiB
Copying blob ddee35dfe011 [========>-------------------------] 863.2KiB / 3.3MiB
Copying blob 8a7a13810b7d [=>-------------------------------] 694.3KiB / 14.8MiB
Copying blob cd4054fd8660 [====================================] 250.0b / 250.0b
Copying blob a2318d6c47ec [=====>-----------------------------] 4.6MiB / 27.8MiB
Copying blob ddee35dfe011 [====================================] 3.3MiB / 3.3MiB
Copying blob 8a7a13810b7d [====>------------------------------] 2.3MiB / 14.8MiB
Copying blob cd4054fd8660 done  
Copying blob a2318d6c47ec [=========>-------------------------] 7.8MiB / 27.8MiB
Copying blob ddee35dfe011 done  
Copying blob 8a7a

Show the `crop` help:

In [4]:
podman run --rm -it --env=PYTHONPATH=/app localhost/crop:latest python -m app --help

Usage: app.py [OPTIONS]

  Crops a STAC Item asset defined with its common band name

Options:
  --input-item TEXT  STAC Item URL or staged STAC catalog  [required]
  --aoi TEXT         Area of interest expressed as a bounding box  [required]
  --epsg TEXT        EPSG code  [required]
  --band TEXT        Common band name  [required]
  --help             Show this message and exit.


## Test the crop step in the container

Crop the green band asset:

In [5]:
podman run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/crop:latest \
    python \
    -m \
    app \
    --aoi \
    "-121.399,39.834,-120.74,40.472" \
    --band \
    green \
    --epsg \
    "EPSG:4326" \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A

2024-12-10 16:46:03.098 | INFO     | __main__:crop:69 - Read S2B_10TFK_20210713_0_L2A from https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
2024-12-10 16:46:03.126 | INFO     | __main__:crop:72 - Read asset green from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-12-10 16:46:05.390 | INFO     | __main__:crop:90 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B03.tif
2024-12-10 16:46:24.680 | INFO     | __main__:crop:112 - Write crop_green.tif
2024-12-10 16:46:27.663 | INFO     | __main__:crop:115 - Done!


Crop the nir band asset:

In [6]:
podman run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/workspace/mastering-app-package/runs,target=/runs \
    --workdir=/runs \
    --read-only=true \
    --user=1001:100 \
    --rm \
    --env=HOME=/runs \
    --env=PYTHONPATH=/app \
    localhost/crop:latest \
    python \
    -m \
    app \
    --aoi \
    "-121.399,39.834,-120.74,40.472" \
    --band \
    nir \
    --epsg \
    "EPSG:4326" \
    --input-item \
    https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A

2024-12-10 16:46:29.994 | INFO     | __main__:crop:69 - Read S2B_10TFK_20210713_0_L2A from https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A
2024-12-10 16:46:30.022 | INFO     | __main__:crop:72 - Read asset nir from https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B08.tif
2024-12-10 16:46:32.256 | INFO     | __main__:crop:90 - Crop https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/10/T/FK/2021/7/S2B_10TFK_20210713_0_L2A/B08.tif
2024-12-10 16:46:47.548 | INFO     | __main__:crop:112 - Write crop_nir.tif
2024-12-10 16:46:50.592 | INFO     | __main__:crop:115 - Done!


List the outputs:

In [7]:
tree ${RUNTIME}

/workspace/mastering-app-package/runs
├── crop.log
├── crop_green.tif
└── crop_nir.tif

0 directories, 3 files
